# Problem Definition

https://en.wikipedia.org/wiki/German_tank_problem

Lets set up the problem. In WW2, the allied forces are trying to estimate the number of german tanks. One idea is to use the power of statistics to complete this. Lets assume that during manufacturing, each tank got a serial number assigned to it. The serial numbers increase by sequence. To simplify this, the first tank is `1`, then `2`, and so forth. For `n` tanks, the serial number of the nth tank is `n`.

So as the allied forces, we are able to capture a few of these tanks. Can we use the observed serial numbers to then estimate the number of tanks actually manufactured?

We have to make a few simple assumptions:

- First, as already stated, we assume that the serial numbers assigned to each tank is given in a numerically increasing order. (see above).
- Second, we assume tanks are distributed in a uniform fashion. So the capture tanks are uniformly sampled by the manufacturing process. So the serial number observed is indpenent of each other.

Before we go through the math, lets generate a sequence of `n` tanks.

In [23]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [76]:
N = 122
def manufacture_n_tanks(n):
    return np.arange(0,n, 1)
tank_population = manufacture_n_tanks(N)

Now lets "capture" a few of them

In [75]:
import scipy
k = 15
def capture_k_tanks(tank_population, k):
    return np.random.choice(tank_population, replace=False, size=k)
tank_sample = capture_k_tanks(tank_population, k)
print tank_sample

[  4  27   6  73  14  57 120  91  11  58  32  84  18 109  12]


Some notation to consider:
- $N$ is the number of tanks in the total population. This is also the maximum value that can be observed
- $k$ is the sample size (ie. tank_sample)
- $m$ is the maximum serial number in the tank sample. 

There are some basic constraints:
- $k \leq N$ - since the sample size can never exceed the population
- $k \leq m \leq N$ - since the largest number in the sample can never be less than the number of samples and can never be greater than the population (since N is the max number)

To estimate the size of the population, lets consider the Frequentist method. Later on we can try the Bayesian method (TODO)

## Frequentist Method

The intuition of the point estimate is to take the maximum observation in the sample scaled by the number of samples. This is derived by the Maximum Likelihood Estimation.

First, consider a number $m$ s.t. $k \leq m \leq N$ and the probability that the number is the largest 


Let $X_i ~ U(!,N)$ and $X= (X_1,...,X_k)$. Each sample of $X_i$ is iid. Assume we are sampling from a discrete uniform distribution.


$$ P(X=x|N) = \frac{1}{N-1} $$

If we consider then the MLE, ie the joint distribution of iid samples $X$. 

$$ P(X) = P(X=X_1) \bullet P(X=X_2) ... \bullet P(X=X_N)  = \prod_{i} P(X=X_i|N) $$
The likelihood function is
$$ L(X|N) = \arg\max_{N} \prod_{i=1}^{k} \frac{1}{N-1} = (N-1)^{-k}$$ 

So what value of $N$ maximizes that function?

$$ \frac{ d \ L(X|N)}{ d \ N} = -\frac{k}{N-1}^{k-1} < 0$$

Since the derivative is negative, the maximum occurs when $N$ is as large as possible.
From our observed data:

$$ \hat{N} = max(X)=m$$

If we say $X_{(1)} < X_{(2)} < .. X_{(k)}$, then the $max(X)=X_{(k)}$ and $\hat{N}=X_{(k)}$


If we look at the bias of our estimate

$$E(\hat{N}) = E(X_{(k)}) = \sum_{i=0}^{N} X_{(k=i)} P(X_{(k=i)})$$

So what's the probability of $m$?
Given the number of samples $k$ and the population size of $N$, the probability that the random variable $m$ is the maximum of the sample is determined by the possible combinations of the samples. So for up to the value $m$, there are $\begin{pmatrix} m \\ k \end{pmatrix}  $ possible samples to be observed. However, we only care about the few where $m$ is part of that sample, so the difference of $\begin{pmatrix} m-1 \\ k \end{pmatrix}$. In total, there are $\begin{pmatrix} N \\ k \end{pmatrix}  $
$$P(M=m|k,N) =
\begin{cases} 
      0 & m < k \\
      \frac
          {\begin{pmatrix} m \\ k \end{pmatrix}-\begin{pmatrix} m-1 \\ k \end{pmatrix}}
          {\begin{pmatrix} N \\ k \end{pmatrix}  } & k \leq m \leq N \\
      0 & m > N
\end{cases}$$

Lets simplify the piecewise function a bit.

$$  \frac
  {\begin{pmatrix} m \\ k \end{pmatrix}-\begin{pmatrix} m-1 \\ k \end{pmatrix}}
  {\begin{pmatrix} N \\ k \end{pmatrix}  } = 
  \frac{\frac{m!}{k!(m-k)!} - \frac{(m-1)!}{k!(m-1-k)!}}
      {\frac{N!}{k!(N-k)!}} = 
  \frac{\frac{m!-(m-k)(m-1)!}{(m-k)!}}
      {\frac{N!}{(N-k)!}} =
  \frac{\frac{m!-m(m-1)! + k(m-1)!}{(m-k)!}}
      {\frac{N!}{(N-k)!}}   =
  \frac{\frac{k(m-1)!}{(m-k)!}}
      {\frac{N!}{(N-k)!}}   
      $$
      
      


    
$$ E(\hat{N}) = E(m) = \sum_{m=k}^{N} m P(m) = \sum_{m=k}^{N} m \frac{\frac{k(m-1)!}{(m-k)!}}
      {\frac{N!}{(N-k)!}}     = \frac{k(N+1)}{k+1} \neq N $$

We can then see that the estimate for $N$ is biased! Lets rewrite the expected value of the estimate to get the unbiased form of 

$$ E(m) = \frac{k(N+1)}{k+1} $$

We can then rewrite the estimate of $N$ as:

$$ E(m)(1 + k^{-1})-1 = E[m(1 + k^{-1})-1] = E[\hat{N}] $$

$$ \hat{N} = m(1+k^{-1})-1 $$

In [78]:
def estimate_n(samples):
    return max(samples)*(1+1/float(len(samples))) - 1
print "Estimate of N from sample data:" , estimate_n(tank_sample)

Estimate of N from sample data: 127.0


We can also compute the confidence interval of our estimates.. but we'll save that for a later day :)